In [1]:
%load_ext autoreload
%autoreload 2
import torch
from GPT import * 
from dataclasses import dataclass

In [14]:
stoidx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 '*': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '/': 11,
 '0': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '4': 16,
 '5': 17,
 '6': 18,
 '7': 19,
 '8': 20,
 '9': 21,
 ':': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 'a': 55,
 'b': 56,
 'c': 57,
 'd': 58,
 'e': 59,
 'f': 60,
 'g': 61,
 'h': 62,
 'i': 63,
 'j': 64,
 'k': 65,
 'l': 66,
 'm': 67,
 'n': 68,
 'o': 69,
 'p': 70,
 'q': 71,
 'r': 72,
 's': 73,
 't': 74,
 'u': 75,
 'v': 76,
 'w': 77,
 'x': 78,
 'y': 79,
 'z': 80,
 '{': 81,
 '|': 82,
 '}': 83,
 'É': 84,
 'à': 85,
 'á': 86,
 'â': 87,
 'æ': 88,
 'ç': 89,
 'è': 90,
 'é': 91,
 'ë': 92,
 'ì': 93,
 'í': 94,
 'ï': 95,
 'ò': 96,
 'ö': 97,
 'ø': 98,
 'ù': 99,
 '–': 100

In [15]:
@dataclass
class config:
    vocab_size: int = len(set(text)) +1
    block_size: int = 16
    n_embed:    int = 12
    n_head:     int = 12
    bias:       bool = False
    dropout:  float = 0.2
    n_layer:    int = 12


In [16]:
with open("Data/dialog.txt","r",encoding="utf-8") as f:
    data = f.read()
text = data.replace("|",":")
len(set(data))
words = sorted([i for i in set(data)])

In [17]:
idxtos = {idx:w for idx,w in enumerate(words)}
stoidx = {w:idx for idx,w in enumerate(words)}
encode = lambda s: [stoidx[i] for i in s]
decode = lambda i: "".join(idxtos[j] for j in i)

In [18]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [19]:
def get_batch(split,batch_size):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - config.block_size, (batch_size,))
    x = torch.stack([data[i:i+config.block_size] for i in ix])
    y = torch.stack([data[i+1:i+config.block_size+1] for i in ix])
    return x, y

In [20]:
model = GPT(config)

In [21]:
eval_iters = 200
max_iters = 2000
eval_interval = 200
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split,32)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [25]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3)
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train',32)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.8677, val loss 4.8328
step 200: train loss 2.6866, val loss 2.8395
step 400: train loss 2.5380, val loss 2.6651
step 600: train loss 2.4550, val loss 2.5884
step 800: train loss 2.4108, val loss 2.5527
step 1000: train loss 2.3945, val loss 2.5312
step 1200: train loss 2.3649, val loss 2.4950
step 1400: train loss 2.3275, val loss 2.4617
step 1600: train loss 2.3148, val loss 2.4403
step 1800: train loss 2.3097, val loss 2.4233


In [32]:
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(context, max_new_tokens=500,blocksize=16)[0].tolist()))



*eramabant:Tomuts angouts big nos youring s war it ibe, rasus wit tongrlll. Oh robk youveeintg and jalt-- Ruf tollld mul nis?
Gerar Hrichsralllccarce's, fes dig se llatled whe inor't, su sple' beal - dethe te, ce. paw've.
Digel Sweat's Pes. I'Aus E.
Golarlt:Wit thamee sofig wes fem a melert. Gemorme wery bime sbampes dom… gicgoa. 1orr, tilt've mome alack, i Mas copt'siaonm wrrellar y a nof nawas, momalata kuat. Hased.
Geryouralt:Hio n I Zhat mofert t6…
Gerer:Thow anat
Hhoous That'd ar ralp non'l
